In [1]:
# Set this variable to true if ffmpeg is installed on your computer
ffmpeg_installed = true

true

In [2]:
using Printf
using Random
using LinearAlgebra
using PyPlot
using PyCall
@pyimport matplotlib.animation as anim

rng = MersenneTwister(18);

In [3]:
# Size of matrix
n = 16
Q0 = rand(rng, n, n)
Λ = diagm(0 => Float64[1.4^(-i) for i=0:n-1])

F = qr(copy(Q0)); Q = F.Q
Λ[n,n] = 1e-1 * Λ[n,n]

A = Q * Λ / Q

16×16 Array{Float64,2}:
  0.22286      0.155273    0.100986   …   0.0752276   0.151283     0.0959951
  0.155273     0.28384     0.101614       0.014642    0.135199     0.0976818
  0.100986     0.101614    0.168777       0.0789483   0.0779655    0.0742056
  0.122288     0.0916859   0.0247263      0.139472    0.107246     0.0513646
  0.154687     0.177238    0.0786283      0.0520992   0.120512     0.167997 
  0.0245992    0.026526    0.0181994  …  -0.0340837   0.0494239    0.08648  
  0.0417553    0.0158115   0.0267961      0.132738    0.0297757    0.0220979
  0.00527576   0.0291958   0.0586396     -0.0262454   0.0186256    0.0905244
  0.00566107   0.0012239   0.0119001     -0.0550903   0.0739205    0.0289141
  0.0299928    0.00463753  0.0504976      0.0169019   0.0224599    0.0112967
 -0.0166038   -0.112728    0.0729674  …   0.203017   -0.0587797   -0.0451388
 -0.0147233    0.0345941   0.114965       0.0492229  -0.00233977   0.0296951
  0.00998046  -0.0148307   0.0687593      0.0295353 

In [4]:
Tk = copy(A)
for k=1:10
    F = qr(Tk); Uk = F.Q; Rk = F.R;
    Tk = Rk * Uk
end

# Print approx. and exact evalues
n_prt = 3
Tkdiag = diag(Tk)
exact = diag(Λ)
println("Tk    ",Tkdiag[1:n_prt])
println("Exact ",exact[1:n_prt])

Tk    [0.999991, 0.714225, 0.510182]
Exact [1.0, 0.714286, 0.510204]


In [5]:
function record_snapshot(A, ims)
    """Saves figure snapshot for the animation"""
    im = imshow(copy(A), interpolation="nearest",
                extent=(0.5,size(A,1)+0.5,size(A,2)+0.5,0.5))
    clim(-8,1)
    push!(ims, PyCall.PyObject[im])
end

function create_animation(A, filename)
    fig = figure()
    ims = []

    Tk = copy(A)
    record_snapshot(log.(abs.(A))/log(10), ims)
    for k=1:70
        Uk, Rk = qr(Tk)
        Tk = Rk * Uk
        record_snapshot(log.(abs.(Tk))/log(10), ims) # This is used to make the movie
    end
    colorbar()
    ani = anim.ArtistAnimation(fig, ims, interval=100, blit=true)
    ani[:save](filename, extra_args=["-vcodec", "libx264", "-pix_fmt", "yuv420p"])
end

# Create animation for QR iteration
if ffmpeg_installed && ( !isfile("QR_iteration.mp4") || !isfile("QR_iteration_unsym.mp4") )

    F = qr(copy(Q0)); Q = F.Q
    A = Q * Λ / Q
    create_animation(A, "QR_iteration.mp4")
    
    A = Q0 * Λ / Q0
    create_animation(A, "QR_iteration_unsym.mp4")    
    
end

In [6]:
using Base64
function html_video(filename)
    open(filename) do f
        base64_video = base64encode(f)
        """<video controls src="data:video/x-m4v;base64,$base64_video">"""
    end
end

html_video (generic function with 1 method)

In [7]:
# Symmetric case
if isfile("QR_iteration.mp4") # Checks whether movie file exists
    display("text/html", html_video("QR_iteration.mp4"))
end

In [8]:
# Unsymmetric case
if isfile("QR_iteration_unsym.mp4") # Checks whether movie file exists
    display("text/html", html_video("QR_iteration_unsym.mp4"))
end